In [1]:
import tushare as ts
import pandas as pd
from pandas import DataFrame
from pandas import concat
from pandas import merge
import numpy as np
import matplotlib.pyplot as plt
import os, sys
import time

# 1、注册接口

In [2]:
#ts.set_token('4a7caeea1e5e4718dac4386b6e4f7bfefdafd2f9634f9e71e6ffaaef')
#ts.set_token('d6fe22750410e124c057a41d1c8031d684b8e0d2f1b96b694db0ea2a') #15112994875
#ts.set_token('ca4d2743af67b24bcbc3b14f09fa349b9877bc893b28706041600a9e') #492207670@qq.com
#ts.set_token('86f7cdc6aad080403432e395cee208218866d4f091063e69d8c8d755') #714553504@qq.com
#ts.set_token('0242bb7d53bef05e3ce8e426c7b4ac86edf30ca779d0b0e48d4a7882') #1755347637@qq.com
#ts.set_token('8f68a2ccc10e31a9d66ae27e6930bdb8cb99a6eb75b8d5e8da0a495b')  #26098495@qq.com
#ts.set_token('9f5585ac3e8a5fd2e4be500723102060c609881d22e2b9f9436d3409')  #2364480207@qq.com
ts.set_token('b1b07de72441487cc9bdced5c94fab3765605559a50106e3e621a95c')  #763690782@qq.com
#ts.set_token('01e7c939be853b6e8cc8b2cfb19facd42733e68062485b5d8c8a3a38')  #3509036093@qq.com
#ts.set_token('52341d49b9b7616a7f98976bcb41d29d21e723a68015afd6c97f781b')  #13433245820
#ts.set_token('a06020cba02aa5cf32bc461b4e529af5852de1e7c19e5a22a9f7c118')  #roelwf134@163.com

#ts.set_token('029060c40e4a6b9b2ffc3e55a18f6690b9e3ba9c77ab8a1ece015bda')  #2978373708@qq.com
#ts.set_token('e6215c86b05f42be90a84255b65924c7e3e64cd8b189f09de97a9a90')  #huelw124@sina.com
#ts.set_token('89f9642565ae1134ca836fe089382335b46b880a626c2ec42ade6d87')  #1516836216@qq.com
#ts.set_token('2b0108ead201cfff4f868f17f5f4269ac9617d740e24aea8c7ae5983') #holdetf@sina.com



In [3]:
pro = ts.pro_api()

In [4]:
start = time.time()

# 2、更新基础信息

In [5]:
# 更新股票基础信息 
def to_stock_list():
    df = pro.stock_basic(list_status='L', fields='ts_code,symbol,name,area,industry,list_date,fullname,market,exchange,list_status,is_hs')
    df = df.to_excel('../JN_DataWarehouse/stock_analysis/TuShare/stock_list.xls')

In [6]:
#更新交易日历
def to_cal():
    df = pro.trade_cal(exchange='', start_date='20010101', end_date='20290101', fields='exchange,cal_date,is_open,pretrade_date')
    df = df.to_excel('../JN_DataWarehouse/stock_analysis/TuShare/cal.xls')

In [7]:
# 上市公司基本信息
def to_stock_company():
    df = pro.stock_company()
    df = df.to_excel('../JN_DataWarehouse/stock_analysis/TuShare/company.xls')

In [8]:
to_stock_list()
to_cal()
to_stock_company()


# 3、更新每日stock信息

In [8]:
# 读取日历
def read_cal():
    df = pd.read_excel('../JN_DataWarehouse/stock_analysis/TuShare/cal.xls', index_col=False)
    return df


# 读取日历，按时间段
def read_cal_t(start_date, end_date):
    df = pd.read_excel('../JN_DataWarehouse/stock_analysis/TuShare/cal.xls', index_col=False)
    df_cal_t = df.query("is_open == 1 and cal_date >= "+start_date+" and cal_date <= "+end_date+"")[['cal_date']]  # query 查询语句
    return df_cal_t

In [9]:
# 读取year
def read_year(year):
    df = pd.read_csv('../JN_DataWarehouse/stock_analysis/TuShare/'+year+'.csv', index_col=False)
    return df

# 更新 year
def to_year(df, year):
    df = df.to_csv('../JN_DataWarehouse/stock_analysis/TuShare/'+year+'.csv')

# 更新 year
def to_newyear(df):
    df = df.to_csv('../JN_DataWarehouse/stock_analysis/TuShare/new_year.csv')

In [10]:
# 生成日数据按年合并
def build_daily(year):
    
    start_date = year+'0101'
    end_date = year+'1231'
    
    df_cal_t = read_cal_t(start_date, end_date)
    count = int( df_cal_t.count() )
    print(count)
    
    df_year = pro.daily(start_date='?', end_date='?')
    print(df_year)
    for r in zip(df_cal_t['cal_date']):
        #print(r[0])
    
        df_daily = pro.daily(start_date=r[0], end_date=r[0])
    
        df_year = df_year.append(df_daily,ignore_index = True)         
    
        time.sleep(1)
    
        count= count - 1
        if count==0:
            break
    
    to_year(df_year, year)
    print('ok!')

In [11]:
# 刷新最细stock数据
build_daily('2020')

243
Empty DataFrame
Columns: [ts_code, trade_date, open, high, low, close, pre_close, change, pct_chg, vol, amount]
Index: []
ok!


In [12]:
print('finish:%.4f'%(time.time()-start))

finish:453.2268


In [13]:
df_cal_t = read_cal_t('20200101', '20201231')
count = int( df_cal_t.count() )
print(count)    


243


# 4、读取XX公司日数据，按时间段

In [14]:
# 读取year
def read_year(year):
    df = pd.read_csv('../JN_DataWarehouse/stock_analysis/TuShare/'+year+'.csv' ,index_col=False)
    return df

In [15]:
def build_stock_newyear():
    years = ['2001','2002','2003','2004','2005','2006','2007','2008','2009','2010',
             '2011','2012','2013','2014','2015','2016','2017','2018','2019','2020']
    df = read_year('2011')
        
    for x in years:
        df1 = read_year(x)
        df = concat([df,df1],ignore_index=True).drop_duplicates() 
    
    del df['Unnamed: 0']
    
    to_newyear(df)
    
    return df

In [16]:
%%time
df_stock_daily = build_stock_newyear()
df_stock_daily.info

Wall time: 7min 13s


<bound method DataFrame.info of            ts_code  trade_date   open   high    low  close  pre_close  change  \
0        002007.SZ    20110104  48.51  48.69  47.42  48.13      48.39   -0.26   
1        000088.SZ    20110104   6.15   6.18   6.09   6.17       6.13    0.04   
2        000068.SZ    20110104   6.01   6.30   5.94   6.30       6.00    0.30   
3        600167.SH    20110104   9.62   9.78   9.58   9.74       9.63    0.11   
4        002418.SZ    20110104  22.05  22.64  22.01  22.58      22.00    0.58   
...            ...         ...    ...    ...    ...    ...        ...     ...   
9955240  300457.SZ    20201130  28.72  29.16  28.48  28.69      28.82   -0.13   
9955241  300515.SZ    20201130  11.95  12.05  11.65  11.87      11.88   -0.01   
9955242  603968.SH    20201130  17.85  17.99  17.45  17.50      17.72   -0.22   
9955243  603633.SH    20201130  10.75  11.02  10.66  10.84      10.73    0.11   
9955244  689009.SH    20201130  72.60  74.66  66.00  71.35      70.99    0.36

In [17]:
df_stock_daily.info

<bound method DataFrame.info of            ts_code  trade_date   open   high    low  close  pre_close  change  \
0        002007.SZ    20110104  48.51  48.69  47.42  48.13      48.39   -0.26   
1        000088.SZ    20110104   6.15   6.18   6.09   6.17       6.13    0.04   
2        000068.SZ    20110104   6.01   6.30   5.94   6.30       6.00    0.30   
3        600167.SH    20110104   9.62   9.78   9.58   9.74       9.63    0.11   
4        002418.SZ    20110104  22.05  22.64  22.01  22.58      22.00    0.58   
...            ...         ...    ...    ...    ...    ...        ...     ...   
9955240  300457.SZ    20201130  28.72  29.16  28.48  28.69      28.82   -0.13   
9955241  300515.SZ    20201130  11.95  12.05  11.65  11.87      11.88   -0.01   
9955242  603968.SH    20201130  17.85  17.99  17.45  17.50      17.72   -0.22   
9955243  603633.SH    20201130  10.75  11.02  10.66  10.84      10.73    0.11   
9955244  689009.SH    20201130  72.60  74.66  66.00  71.35      70.99    0.36